In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# torch.nn包可以进行神经网络的构建
# nn是在autograd基础上进行模型的定义和微分
# nn.module包含着神经网络的层，同时forward（input）方法可以将output进行返回
# torcn.nn是专门为神经网络设计的模块化接口. nn构建于autograd之上，可以用来定义和运行神经网络。

In [3]:
class Net(nn.Module):
    def __init__(self):

        # nn.Module子类的函数必须在构造函数中执行父类的构造函数
        # nn.Module.__init__(self)
        super(Net, self).__init__()
        
        # Pytorch基于nn.Module构建的模型中，只支持mini-batch的Variable输入方式。比如，只有一张输入图片，也需要变成NxCxHxW的
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        
        # 仿射层/全连接层，y = Wx + b
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self, x):
        # 卷积 -> 激活 -> 池化 
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        
        # reshape，‘-1’表示自适应
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size=x.size()[1:]
        num_features=1
        for s in size:
            num_features *= s
        return num_features
            
# 只需要定义一个forward函数，backward会自动生成，可以在forward函数中使用所有tensor操作
# 参数的模型由net.parameters()返回

In [4]:
'''
神经网络的输出结果是这样的
Net (
(conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
(conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
(fc1): Linear (400 -> 120)
(fc2): Linear (120 -> 84)
(fc3): Linear (84 -> 10)
)
'''

'\n神经网络的输出结果是这样的\nNet (\n(conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))\n(conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))\n(fc1): Linear (400 -> 120)\n(fc2): Linear (120 -> 84)\n(fc3): Linear (84 -> 10)\n)\n'

In [14]:
# 网络的可学习参数通过net.parameters()返回，net.named_parameters可同时返回可学习的参数及名称。
net = Net()
params = list(net.parameters())
print(len(params))   # 10

for param in params:
    print(param.size())
    # print(param)
print(params[0].size()) # conv1's .weight: torch.Size([6, 1, 5, 5])

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])
torch.Size([6, 1, 5, 5])


In [16]:
# forward函数的输入和输出都是Tensor。
# 需要注意的是，torch.nn只支持mini-batches，不支持一次只输入一个样本，即一次必须是一个batch。
# 但如果只想输入一个样本，则用 input.unsqueeze(0)将batch_size设为１。
# 例如 nn.Conv2d 输入必须是4维的，形如nSamples × nChannels × Height × Width. 可将nSample设为1，即1 × nChannels × Height × Width

input = Variable(torch.randn(1, 1, 32, 32))
out = net.forward(input)  # net.forward

print(input)
print(out.size())
print(out)
 
net.zero_grad()   # 所有参数的梯度清零
out.backward(torch.randn(1,10))   # 反向传播

tensor([[[[-0.5332,  1.0975,  0.1023,  ..., -0.2556,  0.5774,  1.1490],
          [ 0.1875,  0.9849,  1.3684,  ..., -1.0755,  0.9035, -0.3509],
          [ 0.9821, -0.9314,  0.3585,  ..., -0.6360,  0.4030, -0.7901],
          ...,
          [-1.0882,  0.0960, -0.3340,  ..., -0.1762, -1.5024, -3.0941],
          [-0.0826,  0.8419,  0.0853,  ..., -0.9719,  1.3274,  0.6328],
          [ 1.0482,  0.2472, -0.5855,  ..., -0.9211, -2.1074,  1.2623]]]])
torch.Size([1, 10])
tensor([[ 0.1292,  0.0208,  0.0956,  0.0975, -0.0057, -0.0272, -0.1178,  0.0040,
          0.0899, -0.0236]], grad_fn=<AddmmBackward>)


In [18]:
# 损失函数: nn实现了神经网络中大多数的损失函数，例如nn.MSELoss用来计算均方误差，nn.CrossEntropyLoss用来计算交叉熵损失。
# 已经定义了一个神经网络，处理了输入以及实现了反馈
# 仍未定义代价函数，计算代价，更新网络权重
# 代价函数：接收（输出，目标）对作为输入，计算出输出和目标之间差距作为代价函数输出
output = net(input)
target = Variable(torch.range(1,10))  # a dummy target, for example
criterion = nn.MSELoss()
loss = criterion(output, target)  # loss是一个scalar
print(loss)

"""
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d  
      -> view -> linear -> relu -> linear -> relu -> linear 
      -> MSELoss
      -> loss
"""

tensor(38.4187, grad_fn=<MseLossBackward>)


/home/xiang/anaconda3/envs/pytorch_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  


'\ninput -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d  \n      -> view -> linear -> relu -> linear -> relu -> linear \n      -> MSELoss\n      -> loss\n'

In [12]:
# 当调用loss.backward()时，该图会动态生成并自动微分，也即会自动计算图中参数(Parameter)的导数。
# 运行.backward，观察调用之前和调用之后的grad
net.zero_grad() # 把net中所有可学习参数的梯度清零
print('反向传播之前 conv1.bias的梯度')
print(net.conv1.bias.grad)
loss.backward()
print('反向传播之后 conv1.bias的梯度')
print(net.conv1.bias.grad)

反向传播之前 conv1.bias的梯度
tensor([0., 0., 0., 0., 0., 0.])
反向传播之后 conv1.bias的梯度
tensor([-0.1493,  0.1278, -0.0439, -0.0774,  0.0175,  0.0143])


In [14]:
# 优化器：在反向传播计算完所有参数的梯度后，还需要使用优化方法来更新网络的权重和参数，例如随机梯度下降法(SGD)的更新策略如下：
weight = weight - learning_rate * gradient

# 手动实现
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)# inplace 减法


In [19]:
# 自动实现：torch.optim中实现了深度学习中绝大多数的优化方法，例如RMSProp、Adam、SGD等，更便于使用，因此大多数时候并不需要手动写上述代码。
import torch.optim as optim
#新建一个优化器，指定要调整的参数和学习率
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# 在训练过程中
# 先梯度清零(与net.zero_grad()效果一样)
optimizer.zero_grad() 

# 计算损失
output = net(input)
loss = criterion(output, target)

#反向传播
loss.backward()

#更新参数
optimizer.step()